In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Загрузка данных из файла train.xlsx
file_path = 'train.xlsx'
data = pd.read_excel(file_path)

# Преобразуем столбец с датами в формат datetime
data['dt'] = pd.to_datetime(data['dt'])

# Сортируем данные по дате
data = data.sort_values('dt')

# Создание дополнительных признаков
data['year'] = data['dt'].dt.year
data['month'] = data['dt'].dt.month
data['day'] = data['dt'].dt.day
data['dayofweek'] = data['dt'].dt.dayofweek

# Разделяем данные на обучающую и тестовую выборки (80% на 20%)
X = data[['year', 'month', 'day', 'dayofweek']]
y = data['Цена на арматуру']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Проверка размера выборок
print(f'Размер обучающей выборки: {X_train.shape[0]}')
print(f'Размер тестовой выборки: {X_test.shape[0]}')

# Создание и обучение модели CatBoostRegressor
model = CatBoostRegressor(iterations=2000, learning_rate=0.05, depth=8, silent=True)
model.fit(X_train, y_train)

# Предсказание с помощью обученной модели
predictions = model.predict(X_test)

# Оценка качества модели
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)

print(f'Mean Absolute Error (MAE): {mae:.2f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
print(f'R²: {r2:.2f}')

# Прогноз на следующие 10 недель
last_date = data['dt'].max()
future_dates = [last_date + pd.DateOffset(weeks=i) for i in range(1, 11)]

# Создание признаков для будущих дат
future_data = pd.DataFrame(future_dates, columns=['dt'])
future_data['year'] = future_data['dt'].dt.year
future_data['month'] = future_data['dt'].dt.month
future_data['day'] = future_data['dt'].dt.day
future_data['dayofweek'] = future_data['dt'].dt.dayofweek

# Прогнозирование цен на новые данные
future_predictions = model.predict(future_data[['year', 'month', 'day', 'dayofweek']])

# Создание графика
plt.figure(figsize=(14, 7))
plt.plot(data['dt'][:len(y_train)], y_train, label='Обучающие данные', color='blue', linestyle='-')
plt.plot(data['dt'][len(y_train):], y_test, label='Фактические цены', color='green', linestyle='-')
plt.plot(data['dt'][len(y_train):], predictions, label='Предсказанные цены', color='orange', linestyle='-')
plt.plot(future_dates, future_predictions, label='Прогноз на 10 недель вперёд', color='red', linestyle='-')
plt.title('Цена на арматуру: Фактические и предсказанные значения')
plt.xlabel('Дата')
plt.ylabel('Цена на арматуру')
plt.xticks(rotation=45)
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()